In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows

os.chdir('/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data')
# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['Sample_data',
 'trade_data_profit_loss_data.csv',
 'cal_data_profit_loss_data.csv',
 'ideal_data_profit_loss_data.csv',
 'trade_data_defence_profit_loss_data.csv',
 'cal_data_defence_profit_loss_data.csv',
 'ideal_data_defence_profit_loss_data.csv']

In [51]:
trade_data = pd.read_csv("trade_data_defence_profit_loss_data.csv")
ideal_data = pd.read_csv("ideal_data_defence_profit_loss_data.csv")
cal_data = pd.read_csv("cal_data_defence_profit_loss_data.csv")

In [52]:
trade_data.columns

Index(['Unnamed: 0', 'Time Stamp', 'Open', 'High', 'Low', 'Close',
       'SellersLevel4', 'SellersLevel3', 'SellersLevel2', 'SellersLevel1',
       'SellersLevel0', 'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2',
       'BuyersLevel3', 'BuyersLevel4', 'BuyersInControlTime',
       'BuyersInControlPrice', 'SellersInControlTime', 'SellersInControlPrice',
       'Entry', 'Defence', 'Stop', 'Target', 'defence_target', 'D_Profit',
       'D_Loss'],
      dtype='object')

In [25]:
trade_data.shape, ideal_data.shape, cal_data.shape

((505221, 27), (505221, 27), (505221, 27))

In [29]:
trade_data.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
ideal_data.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
cal_data.drop(columns=["Unnamed: 0"], axis=1, inplace=True)

In [30]:
trade_data.shape, ideal_data.shape, cal_data.shape

((505221, 26), (505221, 26), (505221, 26))

In [31]:
trade_data['Time Stamp'] = pd.to_datetime(trade_data['Time Stamp'])
ideal_data['Time Stamp'] = pd.to_datetime(ideal_data['Time Stamp'])
cal_data['Time Stamp'] = pd.to_datetime(cal_data['Time Stamp'])

In [32]:
s = "_T"
trade_data.columns = [col + s if col in ['Entry', 'Defence','Stop', 'Target', 'Profit', 'Loss', 'defence_target'] else col for col in trade_data.columns]
s = "_C"
cal_data.columns = [col + s if col in ['Entry', 'Defence','Stop', 'Target', 'Profit', 'Loss', 'defence_target'] else col for col in cal_data.columns]
s = "_Ideal"
ideal_data.columns = [col + s if col in [ 'Target', 'Profit', 'Loss', 'defence_target'] else col for col in ideal_data.columns]



In [33]:
trade_data.shape, ideal_data.shape, cal_data.shape

((505221, 26), (505221, 26), (505221, 26))

In [43]:
trade_data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry_T', 'Defence_T',
       'Stop_T', 'Target_T', 'Profit_T', 'Loss_T', 'defence_target_T'],
      dtype='object')

In [35]:
import pandas as pd

def prepare_final_data(cal_data, ideal_data, trade_data):
    # List of columns to drop
    columns_to_drop = [
        'Time Stamp', 'Open', 'High', 'Low', 'Close',
        'SellersLevel4', 'SellersLevel3', 'SellersLevel2',
        'SellersLevel1', 'SellersLevel0', 'BuyersLevel0',
        'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
        'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
        'SellersInControlTime', 'SellersInControlPrice'
    ]

    # Drop unnecessary columns from cal_data
    cal_data.drop(columns=[col for col in columns_to_drop if col in cal_data.columns], inplace=True)

    # Drop unnecessary columns from ideal_data
    ideal_columns_to_drop = columns_to_drop + ['Entry', 'Defence', 'Stop']
    ideal_data.drop(columns=[col for col in ideal_columns_to_drop if col in ideal_data.columns], inplace=True)

    # Rename columns in trade_data if they exist
    # Rename Target column in ideal_data if it exists

    # Concatenate cal_data and ideal_data to the left of trade_data
    final_data = pd.concat([cal_data, ideal_data], axis=1)
    final_data = pd.concat([trade_data, final_data], axis=1)

    return final_data

# Example usage:
# final_data = prepare_final_data(cal_data, ideal_data, trade_data)



In [36]:
Final_data = prepare_final_data(cal_data, ideal_data, trade_data)

In [37]:
# Define the new column order
new_column_order = [
    'Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
    'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
    'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
    'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
    'SellersInControlTime', 'SellersInControlPrice', 'Entry_T', 'Entry_C',
    'Defence_T', 'Defence_C', 'Stop_T', 'Stop_C', 'Target_T', 'Target_C',
    'Target_Ideal', 'defence_target_T', 'defence_target_C', 'defence_target_Ideal',
    'Profit_T', 'Loss_T', 'Profit_C', 'Loss_C', 'Profit_Ideal', 'Loss_Ideal'
]

# Reorder the columns
Final_data = Final_data[new_column_order]

# Display the updated DataFrame
print(Final_data.columns)

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry_T', 'Entry_C',
       'Defence_T', 'Defence_C', 'Stop_T', 'Stop_C', 'Target_T', 'Target_C',
       'Target_Ideal', 'defence_target_T', 'defence_target_C',
       'defence_target_Ideal', 'Profit_T', 'Loss_T', 'Profit_C', 'Loss_C',
       'Profit_Ideal', 'Loss_Ideal'],
      dtype='object')


In [39]:
Final_data = Final_data.reset_index(drop=True)


In [40]:
Final_data.shape

(505221, 37)

In [41]:
Final_data.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Machine_Learning_data.csv", index=False)

In [ ]:
trader_defence_P_L = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data/trade_data_defence_profit_loss_data.csv")
cal_defence_P_L = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data/cal_data_defence_profit_loss_data.csv")
ideal_defence_P_L = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data/ideal_data_defence_profit_loss_data.csv")

In [ ]:
import pandas as pd

def rename_and_drop_columns(trader_df, cal_df, ideal_df):
    # Rename columns
    trader_df = trader_df.rename(columns={'D_Profit': 'T_D_Profit', 'D_Loss': 'T_D_Loss'})
    cal_df = cal_df.rename(columns={'D_Profit': 'C_D_Profit', 'D_Loss': 'C_D_Loss'})
    ideal_df = ideal_df.rename(columns={'D_Profit': 'I_D_Profit', 'D_Loss': 'I_D_Loss'})

    # Drop 'Unnamed: 0' columns
    trader_df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
    cal_df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
    ideal_df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)

    return trader_df, cal_df, ideal_df


In [ ]:
trader_defence_P_L, cal_defence_P_L, ideal_defence_P_L = rename_and_drop_columns(trader_defence_P_L, cal_defence_P_L, ideal_defence_P_L)

In [ ]:
trader_filtered.shape, cal_filtered.shape, ideal_filtered.shape

((8410, 29), (8410, 29), (8410, 29))

In [ ]:
def concatenate_renamed_columns(trader_df, cal_df, ideal_df):
    # Extract renamed columns
    trader_renamed = trader_filtered[['T_D_Profit', 'T_D_Loss']]
    cal_renamed = cal_filtered[['C_D_Profit', 'C_D_Loss']]
    ideal_renamed = ideal_filtered[['I_D_Profit', 'I_D_Loss']]

    # Concatenate the renamed columns
    concatenated_df = pd.concat([trader_renamed, cal_renamed, ideal_renamed], axis=1)
    return concatenated_df

In [ ]:
Temp_data = concatenate_renamed_columns(trader_defence_P_L, cal_defence_P_L, ideal_defence_P_L)

In [ ]:
Temp_data.columns

Index(['T_D_Profit', 'T_D_Loss', 'C_D_Profit', 'C_D_Loss', 'I_D_Profit',
       'I_D_Loss'],
      dtype='object')

In [ ]:
Temp_data.shape

(8410, 6)

In [ ]:
Temp_data

,T_D_Profit,T_D_Loss,C_D_Profit,C_D_Loss,I_D_Profit,I_D_Loss
226816,0.00,0.0,0.00,0.0,0.00,0.0
226817,0.00,0.0,0.00,0.0,0.00,0.0
226818,69.75,0.0,69.75,0.0,69.75,0.0
226819,69.75,0.0,69.75,0.0,69.75,0.0
226820,542.00,0.0,542.00,0.0,542.00,0.0
...,...,...,...,...,...,...
235221,0.00,0.0,0.00,0.0,0.00,0.0
235222,0.00,0.0,0.00,0.0,0.00,0.0
235223,0.00,0.0,0.00,0.0,0.00,0.0
235224,0.00,0.0,0.00,0.0,0.00,0.0


In [ ]:
Temp_data.reset_index(drop=True, inplace=True)

In [ ]:
Temp_data.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Final_visualization_data/one_month_visualize_Defence_Profit_loss_data.csv",index=False)

In [ ]:
Final_data.shape

(8409, 37)

In [ ]:
Final_data.head(2)

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry_T,Entry_C,Defence_T,Defence_C,Stop_T,Stop_C,Target_T,Target_C,Target_Ideal,defence_target_T,defence_target_C,defence_target_Ideal,Profit_T,Loss_T,Profit_C,Loss_C,Profit_Ideal,Loss_Ideal
0,2021-10-01 00:05:00,4619.0,4621.00,4618.75,4619.25,4622.5,4621.25,4620.25,4620.0,4619.25,4617.5,4617.75,4618.0,4618.25,4618.5,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
1,2021-10-01 00:10:00,4619.0,4620.25,4618.50,4620.00,4622.5,4621.25,4620.25,4620.0,4619.25,4617.5,4617.75,4618.0,4618.25,4618.5,0,0.0,09/30/2021 21:15,4643.0,4643.0,4643.0,4644.0,4644.0,4645.0,4645.0,4641.0,4641.0,4641.0,4642.5,4642.5,4642.5,71.25,0.0,71.25,0.0,71.25,0.0


In [ ]:
Temp_data.head(2)

,T_D_Profit,T_D_Loss,C_D_Profit,C_D_Loss,I_D_Profit,I_D_Loss
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Final_one_month_data = pd.concat([Final_data, Temp_data], axis=1)

In [ ]:
Final_one_month_data.shape

(8410, 43)

In [ ]:
Final_one_month_data.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Final_visualization_data/one_month_visualize_Final_data_with_Defence_P_L.csv",index=False)